In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
import cv2
import torch.utils.data as data
from glob import glob
from random import shuffle
from PIL import Image
import random
from tqdm import tqdm
from pred import predprob
from skimage import io 

In [2]:
import sys
paths = sys.path
sys.path.append('/home/yuyue/yuyue/Synchronized-BatchNorm-PyTorch-master')

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((512, 512)),
#         RandomCrop((512, 512)),
        transforms.RandomHorizontalFlip(),   # horizontal flip
        transforms.RandomVerticalFlip(),   # vertival flip
        transforms.ColorJitter(0.2,0.2,0.2,0.04),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # mean, std
    ]),
    'val': transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
}

In [5]:
df_train = pd.read_csv('/data/Pathology/SPIE/training_set/breastpathq/datasets/train_labels.csv') 
df_val = pd.read_csv('/data/Pathology/SPIE/training_set/breastpathq/datasets/val_labels.csv') 
df = df_train.append(df_val)

In [6]:
def make_dataset(paths, extension, df):
    images = []
    for p in paths:
        if ('.'+extension) in p:
            slide = p.split('/')[-1].split('_')[0]
            slide = int(slide)
            rid = p.split('/')[-1].split('_')[1].split('.')[0]
            rid = int(rid)
            score = df[(df['slide']==slide) & (df['rid']==rid)]['y'].tolist()[0]
           #if score != 0:
            #print(p,float(score))
            images.append([p, float(score)])
    shuffle(images)
    return images

In [7]:
class SPIE_dataset(data.Dataset):
    def __init__(self, dirs, loader, extension, transform=None, train=True):
        self.samples = make_dataset(dirs, extension, df)
        if len(self.samples) == 0:
            raise(RuntimeError("no files in %s" % dirs))
        self.loader = loader
        self.transform = transform
        self.train=train
        
    def __getitem__(self, index):
        path, target = self.samples[index]
        sample = self.loader(path)
        sample = Image.fromarray(sample)
        #target = torch.tensor(target).long()
        if self.transform:
            sample = self.transform(sample)
        if self.train:
            return sample, target,path
        else:
            return sample, target, path
        #print('target:',target)
    def __len__(self):
        return len(self.samples)

In [8]:
df_hard = pd.read_csv('/data/yuyue/SPIE/hard_example.csv')
hard_example = list(df_hard.path)

In [9]:
train_samples = glob("/data/Pathology/SPIE/training_set/breastpathq/datasets/train/*.tif")+hard_example
val_samples = glob("/data/Pathology/SPIE/training_set/breastpathq/datasets/validation/*.tif")
#test_samples = torch.load( '/data/AlgProj/ydx/ydx/zhongshan/datapath/20190410_4_cls/test_444.pth')

In [10]:
train_dataset = SPIE_dataset(train_samples, io.imread, 'tif', transform=data_transforms['train'])
val_dataset = SPIE_dataset(val_samples, io.imread, 'tif', transform=data_transforms['val'])
#test_dataset = Rose_dataset(test_samples, Image.open, 'jpg', transform=data_transforms['test'], train=False)
image_datasets = {'train':train_dataset, 'val':val_dataset}
dataloaders = {"train": torch.utils.data.DataLoader(image_datasets["train"], batch_size=32,
                                             shuffle=True, num_workers=16),
               "val": torch.utils.data.DataLoader(image_datasets["val"], batch_size=8,
                                             shuffle=True, num_workers=4)}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [11]:
def save_model(model, epoch):
    save_dir = "/data/yuyue/SPIE/model_weight/"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    state_dict = model.module.state_dict()
    pth = os.path.join(save_dir, "densenet169_512_0627_sigmoid_hard.pth")
    torch.save(state_dict, pth)

def train_model(model, criterion1,criterion2, optimizer, scheduler, num_epochs=25):
    since = time.time()
    model = DataParallel(model)
   # best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 100.0
    best_epoch = 0
              
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            epoch_labels = []
            epoch_outputs = []
            # Iterate over data.
#             count  = 0
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels,path = data
                #print('labels.type=',labels.type())
                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()
                
                # forward
                outputs = model(inputs)
                
                outputs = nn.Sigmoid()(outputs[:,0])
                #outputs = outputs.unsqueeze(1)
                #print(outputs.shape)
                outputs_normalize = nn.Softmax(dim=0)(outputs)
                label_normalize = nn.Softmax(dim=0)(labels)
                log_outputs_normalize = torch.log(outputs_normalize)
                
                
#                 labels_KL = torch.zeros((len(labels),2))
#                 labels_KL[:,0] = labels
#                 labels_KL[:,1] = 1-labels
                
                # _, preds = torch.max(outputs.data, 1) # pred值为output中最大值的位置（0是neg,1是pos）
                labels = labels.float()                

                loss1 = criterion1(outputs, labels)
                loss2 = criterion2(log_outputs_normalize, label_normalize.float())
                
                loss = loss1+10*loss2
                #print("outputs=",outputs)
                #print("labels=",labels)
                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                    
                for n in range(len(labels)):
                    #print(labels[n].data.cpu().item())
                    pred_value = outputs[n].data.cpu().item()
                    epoch_labels.append(labels[n].data.cpu().item())
                    epoch_outputs.append(pred_value)
                #print(epoch_labels)
                # statistics
                running_loss += loss.item()* inputs.size(0)
                #running_corrects += torch.sum(preds == labels.data)
#                 count += 1
#                 if count%100 == 0:
#                     print("batch %d:" % count)
#                     print(phase+"_loss:", loss.data[0])
#                     print(phase+"_acc:", torch.sum(preds == labels.data)/len(labels.data))

            p_k = predprob(np.array(epoch_labels),np.array(epoch_outputs))
            epoch_loss = running_loss / dataset_sizes[phase]
            #epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f}'.format(
                phase, epoch_loss))
            print('p_k = ',p_k)
#             if phase == "train":
#                 save_model(model, epoch)

#             # deep copy the model
#             if phase == 'val' and epoch_acc > best_acc:
#                 best_acc = epoch_acc
#                 best_model_wts = copy.deepcopy(model.state_dict())

            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_epoch = epoch
                save_model(model, epoch)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    #print('Best val Acc: {:4f}'.format(best_acc))
    print('Best val epoch: {:4f}'.format(best_epoch))

    # load best model weights
#     model.load_state_dict(best_model_wts)
    return model

In [12]:
from torch.nn import DataParallel
from densenet import densenet169
model_ft = densenet169(pretrained=True)
num_ftrs = model_ft.classifier.in_features
model_ft.classifier = nn.Linear(num_ftrs, 1)
#model_ft.add_module("sigmoid", module=nn.Sigmoid())
from sync_batchnorm import convert_model
model_ft = convert_model(model_ft)

# load pretrained model
model_ft.load_state_dict(torch.load("/data/yuyue/SPIE/model_weight/densenet169_512_0627_sigmoid_hard.pth"))
# print("pretrained model loaded")

if use_gpu:
    model_ft = model_ft.cuda()

criterion1 = nn.MSELoss()
criterion2 = nn.KLDivLoss()
# Observe that all parameters are being optimized
# do not forget to change learning rate
# optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9,weight_decay=1e-3)
optimizer_ft = optim.Adam(model_ft.parameters(), lr=1e-4,weight_decay=1e-4)
# Decay LR by a factor of 0.1 every 5 epochs
exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer_ft, T_max=10, eta_min=0, last_epoch=-1)
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.5)

densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [13]:
model_ft = train_model(model_ft, criterion1,criterion2, optimizer_ft, exp_lr_scheduler,
                       num_epochs=30)

Epoch 0/29
----------


/root/anaconda2/lib/python2.7/site-packages/torch/nn/functional.py:1906: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


train Loss: 0.0109
p_k =  0.9348042578537075
val Loss: 0.1053
p_k =  0.7706659920308646
Epoch 1/29
----------
train Loss: 0.0086
p_k =  0.944588995805864
val Loss: 0.0261
p_k =  0.9160078426411992
Epoch 2/29
----------
train Loss: 0.0070
p_k =  0.9517200477955088
val Loss: 0.0286
p_k =  0.8753399531971413
Epoch 3/29
----------
train Loss: 0.0077
p_k =  0.9479338065739966
val Loss: 0.0920
p_k =  0.7793940927202581
Epoch 4/29
----------
train Loss: 0.0071
p_k =  0.9515326859020017
val Loss: 0.0362
p_k =  0.8941243438112707
Epoch 5/29
----------
train Loss: 0.0049
p_k =  0.9631775556490341
val Loss: 0.0500
p_k =  0.8529504775156537
Epoch 6/29
----------
train Loss: 0.0041
p_k =  0.9675732698003976
val Loss: 0.0251
p_k =  0.9103156030611599
Epoch 7/29
----------
train Loss: 0.0040
p_k =  0.9673665884898559
val Loss: 0.0392
p_k =  0.8845107836316488
Epoch 8/29
----------
train Loss: 0.0035
p_k =  0.9713020995467592
val Loss: 0.0328
p_k =  0.891278224021251
Epoch 9/29
----------
train Loss: 

In [10]:
def infer(model, use_gpu=True):
    hard_example = []
    model.train(False)
    running_labels = []
    running_outputs = []
    running_paths = []
    for data in dataloaders['train']:
        inputs, labels,path = data
        if use_gpu:
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
        else:
            inputs, labels = Variable(inputs), Variable(labels)
        #print("labels=",labels)
        
        outputs = model(inputs)
        outputs = nn.Sigmoid()(outputs)
        
        for i in range(len(labels)):
            if abs((outputs[i]-labels[i]).data.cpu()) > 0.2:
                #hard_example.append([path[i],float(outputs[i].data.cpu()),float(labels[i].data.cpu())])   
                hard_example.append(path[i])
        #print("outputs=",outputs)
#         probs, preds = torch.max(outputs.data, 1)
        outputs = list(outputs.cpu().data.numpy())
        labels = list(labels.cpu().data.numpy())
        #paths = list(paths)
        running_labels += labels
        running_outputs += outputs
        running_paths += paths
        
    return np.array(running_outputs), np.array(running_labels),hard_example

In [11]:
from torch.nn import DataParallel
from densenet import densenet169
model_infer = densenet169(pretrained=False)
num_ftrs = model_infer.classifier.in_features
model_infer.classifier = nn.Linear(num_ftrs, 1)
#model_infer.add_module("sigmoid", module=nn.Sigmoid())
from sync_batchnorm import convert_model
model_infer = convert_model(model_infer)
model_infer.load_state_dict(torch.load('/data/yuyue/SPIE/model_weight/densenet169_512_0626_sigmoid_1.pth'))
print("model loaded")
model = model_infer.cuda()
model_infer.eval()
use_gpu = torch.cuda.is_available()
if use_gpu:
    model_infer = model_infer.cuda()
    
model = DataParallel(model)

densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


model loaded


In [12]:
outputs,labels,hard_example= infer(model_infer)

In [13]:
len(hard_example)

90

In [27]:
df_hard = pd.DataFrame(hard_example, columns=["path"]) 

In [29]:
df_hard.to_csv('/data/yuyue/SPIE/hard_example.csv')